In [ ]:
# file paths
sanctions_path = 'DisciplinarySanctions_b4fd5f7f-2575-4911-99a3-2c464103b4561756823407998.xlsx'

accumulation_path = 'AccumulatedCards_b4fd5f7f-2575-4911-99a3-2c464103b4561756823499455.xlsx'

In [5]:
import pandas as pd
from datetime import datetime

def write_suspension_reason(sent_off, event_type, match_event_subtype):
  if sent_off == 'No' and event_type == 'Yellow card':
    return 'Yellow-card accumulation'
  elif sent_off == 'Yes' and event_type == 'Yellow card':
    return 'Second caution'
  else:
    return match_event_subtype
  
def rewrite_club(club, serving_team):
  if type(serving_team) is str:
    return serving_team
  else:
    return club
  
def adjust_for_plurals(value, is_match):
  if is_match:
    if value == 1:
      return 'match'
    else:
      return 'matches'
  else:
    if value == 1:
      return 'month'
    else:
      return 'months'
    
def rewrite_values(value, is_match, is_not_months_for_remaining, return_date):
    if is_not_months_for_remaining:
      return return_date
    else:
      return f'{value} {adjust_for_plurals(value, is_match)}'
    
def rewrite_remaining(value, is_match, return_date):
  if is_match:
    return f'{value} {adjust_for_plurals(value, is_match)}'
  else:
    return_timestamp = pd.to_datetime(return_date).date()
    return return_timestamp

In [6]:
sanctions = pd.read_excel(sanctions_path)

sanctions = sanctions[sanctions['Competition type'].str.contains('L1 Cup|Ontario')]

sanctions = sanctions[sanctions['Case status'].str.contains('CONFIRMED')]

sanctions['Team'] = sanctions.apply(lambda x: rewrite_club(x['Club'], x['Serving team']), axis=1)

sanctions['Reason for Suspension'] = sanctions.apply(lambda x: write_suspension_reason(x['Sent off'], x['Event Type'], x['Match event sub-type']), axis=1)

sanctions['Reason for Suspension'] = sanctions['Reason for Suspension'].apply(lambda x: 'Denying an obvious goalscoring opportunity' if 'Denying an obvious goalscoring opportunity' in x else x)

sanctions['Sanction type'] = sanctions['Sanction type'].map(lambda x: True if x == 'Match suspension' else False)

sanctions['Value'] = sanctions.apply(lambda x: rewrite_values(x['Value'], x['Sanction type'], False, x['Sanction date to (EDT)']), axis=1)

sanctions['Remaining'] = sanctions.apply(lambda x: rewrite_remaining(x['Remaining'], x['Sanction type'], x['Sanction date to (EDT)']), axis=1)

sanctions = sanctions[['Club', 'Competition type', 'Person name', 'Offender type', 'Reason for Suspension', 'Value', 'Remaining']].sort_values(by=['Club'])

sanctions = sanctions.rename(columns={'Competition type': 'Competition', 'Person name': 'Name', 'Offender type': 'Person type', 'Value': 'Total suspended', 'Remaining': 'Total remaining'})

sanctions

,Club,Competition,Name,Person type,Reason for Suspension,Total suspended,Total remaining
18,BURLINGTON SC L2 WOMEN,League2 Ontario (W),WAYNE VIEIRA,Staff,Second caution,2 matches,1 match
44,BURLINGTON SC MEN,League1 Ontario Premier Division (M),GIANLUCA SPADAFORA,Player,"Using offensive, insulting or abusive language...",1 match,1 match
49,BURLINGTON SC U20 MEN EAST,League1 Ontario U-20 League (M),ROBERT GLOD,Player,"Using offensive, insulting or abusive language...",3 matches,3 matches
48,FC LONDON MEN,League1 Ontario Premier Division (M),OBADA ABDALLAH,Player,Yellow-card accumulation,1 match,1 match
6,MASTER'S FA U20 MEN,League1 Ontario U-20 League (M),ALEJANDRO MIRABEL,Player,Violent conduct,3 matches,1 match
7,MASTER'S FA U20 MEN,League1 Ontario U-20 League (M),LUCA CALAMIA,Player,Violent conduct,3 matches,2 matches
8,MASTER'S FA U20 MEN,League1 Ontario U-20 League (M),CRISTIANO VERRILLI,Player,"Using offensive, insulting or abusive language...",3 matches,1 match
11,MASTER'S FA U20 MEN,League1 Ontario U-20 League (M),SALVATORE TEDESCO,Player,Carryover suspension from indoor,8 months,2026-02-20
32,NORTH MISSISSAUGA SC MEN,League1 Ontario Championship Division (M),DANIEL BARKER,Player,Violent conduct,3 matches,1 match
29,NORTH TORONTO U20 WOMEN,League1 Ontario U-20 League (W),DAIRE O'BRIEN,Coach,Second caution,2 matches,2 matches


In [7]:
last_updated = pd.DataFrame({ 'Club': ['Last updated:'], 'Competition': [datetime.today().strftime('%Y-%m-%d')]})

sanctions = pd.concat([sanctions, last_updated], ignore_index=True)

sanctions.to_csv('active_suspensions.csv', index=False)

In [8]:
pending_sanctions = pd.read_excel(sanctions_path)

pending_sanctions = pending_sanctions[pending_sanctions['Competition type'].str.contains('L1 Cup|Ontario')]

pending_sanctions = pending_sanctions[pending_sanctions['Case status'].str.contains('ENTERED')]

pending_sanctions['Pending since'] = pending_sanctions.apply(lambda x: pd.to_datetime(x['In status since (EDT)']).date(), axis=1)

pending_sanctions = pending_sanctions[['Parent Club', 'Person name', 'Offender type', 'Pending since']].sort_values(by=['Pending since'])

pending_sanctions = pending_sanctions.rename(columns={'Parent Club': 'Club', 'Person name': 'Name', 'Offender type': 'Person type'})

last_updated = pd.DataFrame({ 'Club': ['Last updated:'], 'Name': [datetime.today().strftime('%Y-%m-%d')]})

pending_sanctions = pd.concat([pending_sanctions, last_updated], ignore_index=True)

pending_sanctions.to_csv('pending_suspensions.csv', index=False)

pending_sanctions


,Club,Name,Person type,Pending since
0,THE BOROUGH FC,RYAN DENNIS SILVERSTEIN,Player,2025-08-30
1,NORTH TORONTO SOCCER,LUCA DI MARCO,Player,2025-08-30
2,NORTH TORONTO SOCCER,CYRUS ROLLOCKS,Player,2025-09-01
3,Last updated:,2025-09-01,NaN,NaN


In [9]:
yc_accumulation = pd.read_excel(accumulation_path)

yc_accumulation = yc_accumulation[yc_accumulation['Competition type'].str.contains('L1 Cup|Ontario')]

yc_accumulation['yc count'] = yc_accumulation.groupby(['Person name', 'Club', 'Type', 'Competition'])['Person name'].transform('count')

yc_accumulation = yc_accumulation[['Person name', 'Club', 'Type', 'Competition', 'yc count']]

yc_accumulation = yc_accumulation.drop_duplicates(subset=['Person name', 'Club', 'Type', 'Competition'], keep='last')

yc_accumulation = yc_accumulation[(
  ((yc_accumulation['Type'] == 'Team official') & (yc_accumulation['yc count'] >= 2)) |
  ((yc_accumulation['Type'] == 'Player') & (yc_accumulation['yc count'] == 4)) | 
  ((yc_accumulation['Type'] == 'Player') & (yc_accumulation['yc count'] >= 6))
)]

last_updated = pd.DataFrame({ 'Person name': 'Last updated:', 'Club': [datetime.today().strftime('%Y-%m-%d')]})

yc_accumulation = pd.concat([yc_accumulation, last_updated], ignore_index=True)

yc_accumulation = yc_accumulation.rename(columns={ 'Person name': 'Name', 'Type': 'Person type'})

yc_accumulation = yc_accumulation[['Name', 'Club', 'Person type', 'Competition']]

yc_accumulation.to_csv('yc_accumulation.csv', index=False)

yc_accumulation



,Name,Club,Person type,Competition
0,Last updated:,2025-09-01,NaN,NaN
